<a href="https://colab.research.google.com/github/micaelCZ/Paper_Repositorio/blob/main/EscenarioB_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install tensorflow-addons
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall
from keras.callbacks import EarlyStopping



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
# Load the data
url = 'https://raw.githubusercontent.com/micaelCZ/Paper_Repositorio/main/dataset/datasetPreprocesado/Escenario2.csv'
df = pd.read_csv(url)
df = pd.read_csv(url,low_memory=False)
dataframe = pd.read_csv(url,low_memory=False)

In [45]:
# Normalise the data
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df

In [46]:
# Split the data into train and test sets
y = df['label']
label_map = {'AUDIO': 0, 'BROWSING': 1, 'CHAT': 2, 'FILE-TRANSFER' : 3, 'MAIL' : 4, 'P2P' : 5, 'VIDEO' : 6, 'VOIP' : 7}
change_labels = lambda x: label_map[x] if x in label_map else -1
y = df['label'].apply(change_labels)

In [47]:
X = df.iloc[:, 4:-1]
X = dfNormalize(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [48]:
# Reshape the data for Conv1D
X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))


In [49]:
# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [50]:
# Define the model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    keras.layers.MaxPool1D(pool_size=2),
    keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    keras.layers.MaxPool1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(8, activation='softmax')
])

In [51]:

from tensorflow_addons.metrics import F1Score


In [52]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=8, average='macro')])


In [53]:
# Set up EarlyStopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5)


In [54]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop])


Epoch 1/10
202/202 [==============================] - 6s 20ms/step - loss: 1.6094 - accuracy: 0.4093 - precision_2: 0.5317 - recall_2: 0.1291 - f1_score: 0.1771 - val_loss: 1.4180 - val_accuracy: 0.4742 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - val_f1_score: 0.1792
Epoch 2/10
202/202 [==============================] - 4s 20ms/step - loss: 1.4279 - accuracy: 0.4668 - precision_2: 0.6266 - recall_2: 0.2193 - f1_score: 0.2286 - val_loss: 1.3828 - val_accuracy: 0.4997 - val_precision_2: 0.5834 - val_recall_2: 0.3108 - val_f1_score: 0.2602
Epoch 3/10
202/202 [==============================] - 4s 19ms/step - loss: 1.3255 - accuracy: 0.5097 - precision_2: 0.6845 - recall_2: 0.2920 - f1_score: 0.2879 - val_loss: 1.1930 - val_accuracy: 0.5625 - val_precision_2: 0.7651 - val_recall_2: 0.3785 - val_f1_score: 0.3373
Epoch 4/10
202/202 [==============================] - 4s 20ms/step - loss: 1.2168 - accuracy: 0.5737 - precision_2: 0.7478 - recall_2: 0.3507 - f1_score: 0.3784 - val_

In [55]:
# Evaluate the model on the test set
test_loss, test_accuracy, test_precision, test_recall, test_f1 = model.evaluate(X_test, y_test)

# Multiply the metrics by 100 to show them on a 0-100 scale
test_accuracy *= 100
test_precision *= 100
test_recall *= 100
test_f1 *= 100


51/51 [==============================] - 0s 4ms/step - loss: 0.8995 - accuracy: 0.6799 - precision_2: 0.8580 - recall_2: 0.5109 - f1_score: 0.4824


In [56]:
# Print the test set metrics
print('Test set metrics:')
print('  Loss:     {:.3f}'.format(test_loss))
print('  Accuracy: {:.3f}'.format(test_accuracy))
print('  Precision:{:.3f}'.format(test_precision))
print('  Recall:   {:.3f}'.format(test_recall))
print('  F1 Score: {:.3f}'.format(test_f1))


Test set metrics:
  Loss:     0.899
  Accuracy: 67.993
  Precision:85.804
  Recall:   51.088
  F1 Score: 48.240
